### Import bibliotek

In [5]:
from selenium import webdriver
from tqdm import tqdm
import requests
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import re

### Sterownik Selenium

In [7]:
driver = webdriver.Edge(executable_path = 'E:\\magisterka\\edgedriver\\msedgedriver.exe')

C:\Users\filip\AppData\Local\Temp/ipykernel_7520/3942697659.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Edge(executable_path = 'E:\\magisterka\\edgedriver\\msedgedriver.exe')


### Pobranie listy posłów

In [8]:
URL = "https://www.sejm.gov.pl/sejm8.nsf/poslowie.xsp"
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")
divs = soup.find("div", {"id": "contentBody"}).find_all("div", {"class": None})

dep_list, party_list = [], []
for div in divs:
    dep_list.append(div.find("div", {"class": "deputyName"}).text)
    party_list.append(div.find("strong", {"class": None}).text)  
d = {"Name": dep_list, "Party": party_list}

deputies = pd.DataFrame(d)

In [9]:
pd.DataFrame(deputies).to_csv("deputies.csv", encoding="utf-8-sig", index=False, sep=";", header=False)

### Utworzenie listy mówców

In [2]:
URL = "https://www.sejm.gov.pl/sejm8.nsf/wypowiedzi.xsp?view=3"
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")

speakers = []

names = soup.find("ul", {"class" : "category-list"}).find_all("a", href=True)
for name in names:
    speakers.append(name.text)
speakers_encoded = [w.replace(' ', '%20') for w in speakers]

### Pobranie wypowiedzi

In [36]:
from tqdm.notebook import tqdm
import time
main_url = "https://www.sejm.gov.pl/sejm8.nsf/"

sitting_list = []
day_list = []
date_list = []
number_list = []
title_list = []
speaker_list = []
speech_text_list = []

for name in tqdm(speakers):
    name_encoded = name.replace(' ', '%20')
    page_n = 1
    pagination_empty = False
    while(True):
        URL = "https://www.sejm.gov.pl/sejm8.nsf/wypowiedzi.xsp?view=3&mowca={}&page={}".format(name_encoded, page_n)
        page = requests.get(URL)
        soup = BeautifulSoup(page.content, "html.parser")
        table = soup.find("table", {"class": "table border-bottom lista-wyp"})
        body = table.find("tbody")
        pagination = soup.find("ul", {"class" : "pagination"})
        
        if pagination is None: pagination_empty = True
            
        if body.text == "": break
        print("Speaker: {}. Page: {}".format(name, page_n))
            
        trs = body.find_all("tr")
        for tr in tqdm(trs):
            tds = tr.find_all("td")
            
            sitting_list = []
            day_list = []
            date_list = []
            number_list = []
            title_list = []
            speaker_list = []
            speech_text_list = []
            
            sitting_list.append(tds[0].text)
            day_list.append(tds[1].text)
            date_list.append(tds[2].text)
            number_list.append(tds[3].text)
            title_list.append(tds[4].text)
            speaker_list.append(name)
            
            
            speech_url = tds[4].find("a", href=True)['href']
            page = requests.get(main_url+speech_url)
            soup = BeautifulSoup(page.content, "html.parser")
            time.sleep(0.1)
            
            stenogram = soup.find("div", {"class" : "stenogram"})
            speech_p = stenogram.find_all("p", {"class" : None})
            speech_text = ""
            
            for s in speech_p:
                speech_text += s.text
            
            speech_text_list.append(speech_text)
            
            d = {"sitting": sitting_list, "day": day_list, "date": date_list, "number": number_list, "title": title_list, "speaker": speaker_list, "speech_text": speech_text_list}
            pd.DataFrame(d).to_csv("speech_data.csv", encoding="utf-8-sig", index=False, sep=";", mode="a", header=False)
        if pagination_empty:
            break
        page_n += 1

Speaker: Abramowicz Adam. Page: 1



Speaker: Abramowicz Adam. Page: 2



Speaker: Ajchler Zbigniew. Page: 1



Speaker: Ajchler Zbigniew. Page: 2



Speaker: Ajchler Zbigniew. Page: 3


...


Speaker: Zyska Ireneusz. Page: 9



Speaker: Żmijan Stanisław. Page: 1



Speaker: Żmijan Stanisław. Page: 2



Speaker: Żmijan Stanisław. Page: 3



Speaker: Żyżyński Jerzy. Page: 1
